In [ ]:
import pandas as pd
import re
import demoji
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import pandas as pd
from flask import Flask, jsonify
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from
from flask import Flask, Response, jsonify
import os
cwd = os.getcwd()
files = os.listdir(cwd)
print("Files in %r: %s" %(cwd, files))


app = Flask(__name__)


#membaca file csv abusive dan kamus alay

%cd '/Users/irtea/Documents/MY NB/Project/Data Sains BootCamp/Data/Asset Challenge'
df_abusive = pd.read_csv('abusive.csv')
df_alay = pd.read_csv('new_kamusalay.csv',encoding= 'latin-1')


#mengeksport data abusive dan kamus alay ke database

connection=sqlite3.connect('db_gold_challenge.db')
cursorObj = connection.cursor()
df_abusive.to_sql(name='tb_abusive_ir1', con=connection, index=False, if_exists='replace')
df_alay.to_sql(name='tb_alay_ir1', con=connection, index=False, if_exists='replace')

data_abusive = pd.read_sql_query ('''select * from tb_abusive_ir1''',connection)
data_alay = pd.read_sql_query ('''select * from tb_alay_ir1''',connection)

#menyimpan data hasil cleansing ke Database

cursorObj.execute('''
    CREATE TABLE IF NOT EXISTS data_input (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        original_text TEXT,
        clean_text TEXT
    )
''')
connection.commit()

#  FUnction cleansing

def cleansing(sent):
       
    string = sent.lower()
    string = re.sub(r'[^a-zA-Z0-9]',' ', string)
    string = re.sub(r'http\S+', '', string)  
    string = re.sub(r'[^\x00-\x7f]',r'', string)
    string = re.sub('\?', '', string)
    string = string.replace('/na', '')
    string = re.sub(r'www\.[^ ]+', '', string)
    string = re.sub(r'[^a-zA-Z0-9\s]', '', string)
    string = re.sub(r'\b(\w+)\b\s+\b\1\b', r'\1', string)
    string = re.sub(r'(\w)(\1{2,})', r"\1", string)
    string = re.sub(r"\b[a-zA-Z]\b","",string)
    string = re.sub('(s{2,})',' ',string)
    string = demoji.replace(string, '')
    string = string.strip()

    characters_to_remove = ['xf0', 'x9f', 'x8f', 'xe2', 'x80', 'x9c', 'x84', 'xa3', 'x98', 'x86',
                        'xc2', 'xb2', 'x9d', 'xa4', 'xa2', 'x99', 'xab', 'xaa', 'xe1',
                        'xbd', 'xcf', 'xaa', 'xce', 'xb5', 'xcf', 'x81', 'xce', 'xb7',
                        'xb1', 'xaf', 'xc4', 'x93', 'x82', 'xd', 'url','x8e', 'xb6', 'x8e', 'xa7', 'xa5', 'x8d', 'xba',
                        'x91', 'x8c', 'x88', 'x94', 'x85', 'xa6', 'x91', 'xa', 'x92', 'x89', 'xad',
                        'xb3', 'xd0', 'xbc', 'xd1', 'xc6', 'xd0', 'xbcs', 'xa6', 'x83', 'x9b',
                        'xac', 'xef', 'xbc', 'xef', 'xa5', 'xe0', 'xb8', 'xb4', 'x8c', 'xef', 'xa5', 'xe0', 'xb8', 'xb4', 'xef', 'xbc']
        
    for char_sequence in characters_to_remove:
        string = string.replace(char_sequence, '')
        
    string = re.sub("(username|user|url|rt|xf|fx|xe|xa)\s|\s(user|url|rt|xf|fx|xe|xa)","",string)
        
    return string


# Function replace alay

database_alay = dict(zip(data_alay['kata_alay'], data_alay['kata_baku']))

def replace_alay(text, database_alay, compiled_alay_patterns):
    for alay_pattern, baku_word in zip(compiled_alay_patterns, database_alay.values()):
        text = alay_pattern.sub(baku_word, text)
    return text

compiled_alay_patterns = [re.compile(rf'\b{re.escape(alay_word)}\b', flags=re.IGNORECASE) for alay_word in database_alay.keys()]


# Function Sensor Abusive

database_kata_kasar = set(data_abusive['ABUSIVE'])

def sensor_kalimat(teks, database_kata_kasar):
    kata_kata = teks.split()
    kata_kata_bersih = [kata if kata.lower() not in database_kata_kasar else '*' * len(kata) for kata in kata_kata]
    kalimat_bersih = ' '.join(kata_kata_bersih)
    return kalimat_bersih


app.json_encoder = LazyJSONEncoder
swagger_template = {
    "info": {
        "title":  "API Documentation for Data Processing and Modeling",
        "version": "1.0.0",
        "description": "Dokumentasi API untuk Data Processing dan Modeling Tugas Gold Challenge an Irwan"
    },
    "host": "127.0.0.1:5000"
}

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}


swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

@swag_from("/Users/irtea/Documents/MY NB/Project/Data Sains BootCamp/Data/docs/hello_world.yml", methods=['GET'])
@app.route('/', methods=['GET'])
def hello_world():
    json_response = {
        'status_code': 200,
        'description': "Menyapa Hello World",
        'data': "Hello World",
    }

    response_data = jsonify(json_response)
    return response_data


@swag_from("/Users/irtea/Documents/MY NB/Project/Data Sains BootCamp/Data/docs/text_processing.yml", methods=['POST'])
@app.route('/text-processing', methods=['POST'])
def text_processing():

    
    text = request.form.get('text')
    
    
    # Menjalankan proses cleansing
    
    text_cleansing = cleansing(text)
    
    
    # Menjalankan proses mengganti kata alay


    text_cleansing_alay = replace_alay(text_cleansing, database_alay, compiled_alay_patterns)
    
    
    # Menjalankan proses sensor kata abusive


    text_cleansing_alay_sensor = sensor_kalimat(text_cleansing_alay, database_kata_kasar)

    
    # menyimpan hasil cleansing ke database 
    
    entities = (text, text_cleansing_alay_sensor)
    connection=sqlite3.connect('db_gold_challenge.db')
    cursorObj = connection.cursor()
    cursorObj.execute('''
        INSERT INTO data_input (original_text, clean_text)
        VALUES (?, ?)
    ''', entities)
    connection.commit()
        
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses cleansing dan sensor abusive",
        'data': text_cleansing_alay_sensor,
    }
 
    
    response_data = jsonify(json_response)
    
#     connection.close()
    
    return response_data



@swag_from("/Users/irtea/Documents/MY NB/Project/Data Sains BootCamp/Data/docs/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file', methods=['POST'])
def text_processing_file():

    # Upladed file
    file = request.files.getlist('file')[0]

    # Import file csv ke Pandas
    df = pd.read_csv(file, encoding='latin1')
    
    #verifikasi data tweet apakah available 
    
    if 'Tweet' not in df.columns:
        raise ValueError('Column "Tweet" not found')
        
    # menghapus data yang duplicate
#     df = df.drop_duplicates()


    # Ambil teks yang akan diproses dalam format list
    texts = df['Tweet'].to_list()

    # Lakukan cleansing pada teks
    
    df['Tweet clean'] = df['Tweet'].apply(cleansing)
    
    
    # Menjalankan proses mengganti kata alay

    df['Tweet replace alay'] = df['Tweet clean'].apply(lambda tweet: replace_alay(tweet, database_alay, compiled_alay_patterns))

    
    # Menjalankan proses sensor kata abusive


    df['Tweet sensor abusive'] = df['Tweet replace alay'].apply(lambda tweet: sensor_kalimat(tweet, database_kata_kasar))

    
    # menyimpan data ke database
    connection=sqlite3.connect('db_gold_challenge.db')
    cursorObj = connection.cursor()
    kolom_yang_diekspor = ['Tweet', 'Tweet sensor abusive']
    df_subset = df[kolom_yang_diekspor]
    df_subset.to_sql(name='tb_data_file_ir', con=connection, index=False, if_exists='replace')
   
    # mengubah data ke json
    
    cursor = connection.execute("SELECT * FROM tb_data_file_ir")
    data = cursor.fetchall()
    
    json_data = []
    for row in data:
        json_data.append( row[1]) #urutan kolom data tweet yang telah dicleansing
        
    
    json_response = {
        'status_code': 200,
        'description': "File yang sudah diproses cleansing dan sensor abusive",
        'data': json_data,
    }

    response_data = jsonify(json_response)
    
#     connection.close()
    
    return response_data



if __name__ == '__main__':
   app.run()

Files in '/Users/irtea/Documents/MY NB/Project/Data Sains BootCamp/Data/Asset Challenge': ['.DS_Store', 'data 2.csv', 'data.csv', 'README.md', 'citation.bib', 'db_gold_challenge.db', 'datafix.csv', 'new_kamusalay.csv', 'abusive.csv']
/Users/irtea/Documents/MY NB/Project/Data Sains BootCamp/Data/Asset Challenge
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2023 23:31:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 23:31:22] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 23:31:22] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 23:31:22] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 23:31:22] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 23:31:22] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 23:31:23] "GET /docs.json HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 23:31:38] "POST /text-processing HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 23:31:40] "POST /text-processing HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 23:32:32] "POST /text-processing-file HTTP/1.1" 200 -
